<a href="https://colab.research.google.com/github/Uiuran/engineering_exercises/blob/master/FactoryBuilderPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Factory Builder Exercises

Designing a simple misture of Factories, Builder.

Person builder. 

Factory that works altogether with Person builder, by using context in factory code structure.




In [0]:
# Builder and Factories
from enum import Enum

class Person:

  def __init__(self):
    pass
      
  def __str__(self):    
    return str(self.__dict__)

  def __add__(self, person):
    attr = ["info","name","job","position"]
    for a in attr:
      if hasattr(self, a) or hasattr(person, a):                        
        if hasattr(self, a) and hasattr(person, a):          
          v = getattr(self, a) or 0        
          c = getattr(person, a) or 0
          if v and c:  
            if not issubclass(v.__class__,list):
              v = [v]              
            v.insert(-1,c)
            setattr(self, a, v)            
        elif hasattr(person, a):          
          c = getattr(person, a) or 0
          if c:
            setattr(self, a, c)
        elif hasattr(self, a):          
          v = getattr(self, a) or 0
          if v:
            setattr(self, a, v)
        else:
          delattr(self, a)
    return self

  @staticmethod
  def new():
      return PersonBuilder()

class PersonBuilder:
  def __init__(self):
      self.person = Person()
  def build(self):
      return self.person

class PersonInfoBuilder(PersonBuilder):
  def called(self, name):
      self.person.name = name
      return self

class PersonJobBuilder(PersonBuilder):
  def works_as_a(self, job):
      self.person.job = job
      return self

class PersonPositionBuilder(PersonJobBuilder):
  def position(self, position):
      self.person.position = position
      return self

class PersonBuilderFactory:  

  class PersonInfo(Enum):
    FACTORY = 0
    INFO = 1
    JOB = 2
    POSITION = 3  

  def __init__(self, info, builder_type):
    self.info=info
    self.build=builder_type
    self.builder = []

  def __enter__(self):
    return self(self.info, self.build)

  def __exit__(self, exc_type, exc_val, exc_tb):    
    global person
    person = self.builder[0].build()      
    for i in range(1,len(self.builder)):               
      person += self.builder[i].build()
    del self    

  def __call__(self, info, build):
    if build == 1:      
      self.builder.append(PersonInfoBuilder().called(info))
    elif build == 2:
      self.builder.append(PersonJobBuilder().works_as_a(info))
    elif build == 3:
      self.builder.append(PersonPositionBuilder().position(info))    
    else:      
      raise Exception("Not Found")

    return self  


In [0]:
def test_context_factory_builder():
  with PersonBuilderFactory("info 0", 3) as factory:  
    factory("OOK",1)
    factory("AOK",1)
    factory.builder[0].position('dev').works_as_a("programmer")
    factory("attendent",2)
    factory("marketing",2)
    factory("asdfr",1)

  print(person)

test_context_factory_builder()

{'position': 'dev', 'job': ['attendent', 'marketing', 'programmer'], 'name': ['AOK', 'asdfr', 'OOK']}


In [0]:
import copy


class Address:
    def __init__(self, street_address, suite, city):
        self.suite = suite
        self.city = city
        self.street_address = street_address

    def __str__(self):
        return f'{self.street_address}, Suite #{self.suite}, {self.city}'

class Employee:
    def __init__(self, name, address):
        self.address = address
        self.name = name

    def __str__(self):
        return f'{self.name} works at {self.address}'


class EmployeeFactory:

    main_office_employee = Employee("", Address("123 East Dr", 0, "London"))
    aux_office_employee = Employee("", Address("123B East Dr", 0, "London"))

    @staticmethod
    def __new_employee(proto, name, suite):
        result = copy.deepcopy(proto)
        result.name = name
        result.address.suite = suite
        return result

    @staticmethod
    def new_main_office_employee(name, suite):
        return EmployeeFactory.__new_employee(
            EmployeeFactory.main_office_employee,
            name, suite
        )

    @staticmethod
    def new_aux_office_employee(name, suite):
        return EmployeeFactory.__new_employee(
            EmployeeFactory.aux_office_employee,
            name, suite
        )

# main_office_employee = Employee("", Address("123 East Dr", 0, "London"))
# aux_office_employee = Employee("", Address("123B East Dr", 0, "London"))

# john = copy.deepcopy(main_office_employee)
#john.name = "John"
#john.address.suite = 101
#print(john)

# would prefer to write just one line of code
jane = EmployeeFactory.new_aux_office_employee("Jane", 200)
print(jane)
